In [0]:
outpatient = spark.read.parquet("/mnt/bronze/outpatient/outpatient_claims.parquet")

In [0]:
from pyspark.sql.functions import col, to_date, year, when, lit

def standardize_columns(df):
    return df.toDF(*[c.lower() for c in df.columns])
# Clean Outpatient
outpatient_clean = (
    outpatient.dropDuplicates(["DESYNPUF_ID", "CLM_ID"])
    .withColumn("CLM_FROM_DT", to_date(col("CLM_FROM_DT"), "yyyyMMdd"))
    .withColumn("CLM_THRU_DT", to_date(col("CLM_THRU_DT"), "yyyyMMdd"))
    .fillna("unknown")
)

In [0]:
outpatient_clean.write.mode("overwrite").parquet("/mnt/silver/outpatient/outpatient_clean")

In [0]:
from pyspark.sql.functions import col, year, when, to_date, lit

# Outpatient Silver Table
outpatient_silver = outpatient_clean.select(
    "desynpuf_id", "clm_id", "clm_from_dt", "clm_thru_dt",
    "icd9_dgns_cd_1", "hcpcs_cd_1", "admtng_icd9_dgns_cd"
)

In [0]:
outpatient_silver.write.mode("overwrite").parquet("/mnt/silver/outpatient/outpatient_silver")